# Motivation
论文原文：[《Semi-Supervised Classification with Graph Convolutional Networks》](https://arxiv.org/abs/1609.02907)

参考学习地址：
* https://tkipf.github.io/graph-convolutional-networks/
* [【Graph Neural Network】GCN: 算法原理，实现和应用](https://zhuanlan.zhihu.com/p/78624225)
* https://www.zhihu.com/question/54504471/answer/332657604
* https://www.cnblogs.com/pinard/p/6221564.html

代码：
* https://github.com/tkipf/gcn
* https://github.com/shenweichen/GraphNeuralNetwork

GCN是谱图卷积的一阶局部近似，是一个**多层的图卷积神经网络，每一个卷积层仅处理一阶邻域信息，通过叠加若干卷积层可以实现多阶邻域的信息传递**。

<img style="display: block; margin: 0 auto;" src="../../../assets/images/gcn.png" width = "800" height = "400" alt="GCN" align=center />

## 卷积层的传播规则
$$
\begin{align}
H^{(l+1)}=\sigma(\tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}} H^{(l)} W^{(l)})
\end{align}
$$
其中：
* $\tilde{A} = A + I_N$ 是无向图G的邻接矩阵加上自链接（也即每个顶点的所有相邻边和自身加一条环边$I_N$）
* $\tilde{D}$ 是 $\tilde{A}$ 的度矩阵，即$\tilde{D}_{ii}=\sum_j\tilde{A}_{ij}$
* $H^{(l)}$是第$l$层的激活单元矩阵，$H^0=X$
* $W^{(l)}$是每一层的参数矩阵

GCN的每一层通过邻接矩阵 $A$ 和特征矩阵 $H^{(l)}$ 相乘得到每个顶点邻居特征的汇总，然后再乘上一个参数矩阵 $W^{(l)}$ 加上激活函数 $\sigma$ 做一次非线性变换得到聚合邻接顶点特征的矩阵 $H^{(l+1)}$ 。
* 加上单位矩阵$I_N$是因为希望<font color="red">**在进行信息传播的时候顶点自身的特征信息也得到保留**</font>。
* 对邻接矩阵 $\tilde{A}$ 进行归一化操作 $\tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}}$ 是<font color="red">**为了信息传递的过程中保持特征矩阵 $H$ 的原有分布，防止一些度数高的顶点和度数低的顶点在特征分布上产生较大的差异**</font>。且因为$\tilde{D}$ 由 $\tilde{A}$ 计算得到，所以这部分归一化的结果是可以事先算好的
* 上图中通过若干层GCN每个node的特征从$X$变成了$Z$，无论中间有多少层，node之间的连接关系，即$A$，都是共享的

公式中 $\tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}}$ 与对称归一化拉普拉斯矩阵十分类似，而在谱图卷积的核心就是使用对称归一化拉普拉斯矩阵，这也是GCN的卷积叫法的来历。


## 离散卷积
离散卷积的本质是一种加权求和。CNN的卷积本质是利用一个共享参数的过滤器（kernel），**通过计算中心像素点及相邻像素点的加权和来构成feature map实现空间特征的提取**，加权系数也即卷积核的权重系数。CNN中卷积核系数通过随机化初值，然后根据误差函数通过反向传播梯度下降进行迭代优化，进而实现特征提取的作用。GCN的理论一部分工作就是为了引入可优化的卷积参数。

## GCN的Graph
CNN处理的图像或视频中的像素点（pixel），GCN的Graph通常是社交、信息网络中的拓扑图（Non Euclidean Structure），也即数学（图论）中的用顶点和边建立相应关系的拓扑图。而CNN无法直接处理Non Euclidean Structure数据（每个顶点的相邻顶点数目都可能不同，无法用一个同样尺寸的卷积核来进行卷积计算），拓扑图上的空间特征又亟需被提取。

## 提取拓扑空间特征
除了graph convolution外，主流的实现提取拓扑图空间特征的方式：
* [vertex domain（spatial domain）](https://proceedings.mlr.press/v48/niepert16.pdf):把每个顶点相邻的neighbors（receptive field）找出，normalize处理不同数目neighbors的特征。缺点在于每个顶点都要单独计算处理，提取特征效果可能没有CNN好。
<img style="display: block; margin: 0 auto;" src="../../../assets/images/gcn-1.png" width = "800" height = "200" alt="vertex" align=center />

* spectral domain：作为GCN的基础，借助图谱理论实现拓扑图上的卷积操作。

vertex domain分析问题，需要**逐节点（node-wise）**处理，而图结构是非欧式的链接关系，在很多场景上有明显局限性。而spectral domain将问题转换为**“频域”分析**，不再需要node-wise处理。关于spectral graph theory简单概括，**借助于图的拉普拉斯矩阵的特征值和特征向量来研究图的性质**。

### 拉普拉斯矩阵
对于图$G=(V,E)$，其Laplacian矩阵定义为$L=D-A$，其中$L$是Laplacian矩阵，$D$是顶点的度矩阵（对角矩阵），对角线上元素依次为各个顶点的度，$A$是图的邻接矩阵。
<img style="display: block; margin: 0 auto;" src="../../../assets/images/gcn-2.png" width = "800" height = "180" alt="laplacian" align=center />

常用的拉普拉斯矩阵实际有三种：
* Combinatorical Laplacian: $L=D-A$定义的矩阵
* Symmetric normalized Laplacian: $L^{sys}=D^{-1/2}LD^{-1/2}$，应用于大多数GCN论文。这里利用切比雪夫多项式的一阶近似，巧妙地避开了计算代价极高的拉普拉斯矩阵特征分解
* Random walk normalized Laplacian: $L^{rw}=D^{-1}L$


拉普拉斯矩阵的性质：
* 由于$D$和$A$都是对称矩阵，所以拉普拉斯矩阵也是对称矩阵，可以进行特征分解（谱分解）
* 只在中心顶点和一阶相连的顶点上（1-hop neighbor）有非0元素，其余都为0
* 由于为对称矩阵，其所有特征值都为实数。作为半正定矩阵（对于任意非零实向量$X$，若对称矩阵满足$X^TAX\ge 0$，则称$A$为半正定矩阵），对应的$n$个实数特征值大于等于0，即$0=\lambda_1 \le ... \le \lambda_n$，最小的特征值为0。
* 半正定证明：对于任意向量$f$，有：
$$
\begin{align}
f^TLf &= f^TDf-f^TWf \\
 &= \sum_{i=1}^{n}d_if_i^2-\sum_{i,j=1}^nw_{ij}f_if_j \\
 &= \frac{1}{2}(\sum_{i=1}^{n}d_if_i^2 - 2\sum_{i,j=1}^nw_{ij}f_if_j + \sum_{j=1}^{n}d_jf_j^2) \\
 &= \frac{1}{2}\sum_{i,j=1}^nw_{ij}(f_i-f_j)^2 \ge 0
\end{align}
$$

Tips：$d_i=\sum_j w_{ij}$，因为因为节点$i$的度是与它相连的所有边的权重之和

### 谱分解（特征分解）
矩阵可以特征分解的充要条件是**n阶方阵存在n个线性无关的特征向量**，由于拉普拉斯矩阵是半正定矩阵（半正定矩阵本身就是对称矩阵），其性质满足
1. 半正定矩阵的特征值一定非负
2. 实对称矩阵一定是n个线性无关的特征向量，且其特征向量总可以化为两两相互正交的正交矩阵

所以拉普拉斯矩阵的谱分解有：
$$
\begin{align}
A = U\begin{pmatrix}
\lambda_{1} &  &  \\
 & \ddots &  \\
 &  & \lambda_{n}
\end{pmatrix}
U^{-1} = U\begin{pmatrix}
\lambda_{1} &  &  \\
 & \ddots &  \\
 &  & \lambda_{n}
\end{pmatrix}
U^T
\end{align}
$$

其中，$U=(\overrightarrow{u_1},..,\overrightarrow{u_n})$，是**列向量**为单位特征向量的矩阵（i.e. $\overrightarrow{u_l}$是列向量），由于$U$是正交矩阵，满足$UU^T=E$，其中$E$为单位矩阵。特征分解最右边的是特征矩阵的逆，只是拉普拉斯矩阵的性质才可以写成特征矩阵的转置。

### 傅立叶变换
传统的傅里叶变换
$$
F(\omega) = \mathcal{F}[f(t)] = \int f(t)e^{-i\omega t}dt
$$
信号$f(t)$与基函数$e^{-i\omega t}$的积分，$e^{-i\omega t}$是拉普拉斯算子的特征函数（满足特征方程），$\omega$就和特征值有关。

处理Graph问题时，用到了拉普拉斯矩阵$L$，若$V$是其特征向量，根据广义特征方程式，有$LV=\lambda V$。离散积分就是一种内积形式，仿上定义Graph上的傅里叶变换：
$$
F(\omega_l) = \hat{f}(\lambda_l) = \sum_{i=1}^Nf(i)u_l^*(i)
$$

$f$是Graph上的N维向量，$f(i)$与Graph的顶点一一对应，$u_l(i)$表示第$l$个特征向量的第$i$个分量。那么特征值（频率）$\lambda_l$下的，$f$的Graph傅里叶变换就是与$\lambda_l$对应的特征向量$u_l$进行内积运算。（上述内积运算是在复数空间中定义，所以才用了$u_l$的共轭$u_l^*(i)$）

利用矩阵乘法将Graph上的傅里叶变换推广到矩阵形式：
$$
\begin{align}
\begin{pmatrix}
\hat{f}(\lambda_1) \\
\hat{f}(\lambda_2) \\
\vdots \\
\hat{f}(\lambda_N) \\
\end{pmatrix}
=
\begin{pmatrix}
u_1(1) & u_1(2) & \cdots & u_1(N) \\
u_2(1) & u_2(2) & \cdots & u_2(N) \\
\vdots & \vdots & \ddots & \vdots \\
u_N(1) & u_N(2) & \cdots & u_N(N) \\
\end{pmatrix}
\begin{pmatrix}
f(1) \\
f(2) \\
\vdots \\
f(N) \\
\end{pmatrix}
\end{align}
$$
即$f$在Graph上傅里叶变换的矩阵形式为$\hat{f}=U^Tf$

传统傅里叶逆变换是对频率$\omega$求积分：
$$
\mathcal{F}^{-1}[F(\omega)] = \frac{1}{2\pi}\int F(\omega)e^{iwt}d\omega
$$
迁移到Graph上变为对特征值$\lambda_l$求和：
$$
f(i)=\sum_{l=1}^N\hat{f}(\lambda_l)u_l(i)
$$
对应的矩阵形式
$$
\begin{align}
\begin{pmatrix}
f(1) \\
f(2) \\
\vdots \\
\hat{f}(N) \\
\end{pmatrix}
=
\begin{pmatrix}
u_1(1) & u_2(1) & \cdots & u_N(1) \\
u_1(2) & u_2(2) & \cdots & u_N(2) \\
\vdots & \vdots & \ddots & \vdots \\
u_1(N) & u_2(N) & \cdots & u_N(N) \\
\end{pmatrix}
\begin{pmatrix}
\hat{f}(\lambda_1) \\
\hat{f}(\lambda_2) \\
\vdots \\
\hat{f}(\lambda_N) \\
\end{pmatrix}
\end{align}
$$
即$f$在Graph上傅里叶逆变换的矩阵形式为$f=U\hat{f}$

傅里叶变换的本质是：把任意一个函数表示成若干个正交函数（由$sin\omega t$，$cos\omega t$构成）的线性组合。

### 推广卷积
卷积定理：函数卷积的傅里叶变换是函数傅里叶变换的乘积，即对函数$f(t)$与$h(t)$两者的卷积是其函数傅里叶变换乘积的逆变换：
$$
\begin{align}
f*h=\mathcal{F}^{-1}[\hat{f}(\omega)\hat{h}(\omega)]=\frac{1}{2\pi}\int \hat{f}(\omega)\hat{h}(\omega)e^{i\omega t}d\omega
\end{align}
$$
类比到Graph上把傅里叶变换定义带入，$f$与卷积核$h$在Graph上的卷积按下列步骤求：
1. $f$的傅里叶变换$\hat{f}=U^Tf$
2. 卷积核$h$的傅里叶变换写成对角矩阵形式：$\begin{pmatrix}\hat{h}(\lambda_1)&&\\&\ddots&\\&&\hat{h}(\lambda_n)\end{pmatrix}$，$\hat{h}(\lambda_l)=\sum_{i=1}^Nh(i)u_l^*(i)$是根据需要设计的卷积核$h$在Graph上的傅里叶变换
3. 两者的傅里叶变换乘积为$\begin{pmatrix}\hat{h}(\lambda_1)&&\\&\ddots&\\&&\hat{h}(\lambda_n)\end{pmatrix}U^Tf$
4. 再乘以$U$求两者傅里叶变换乘积的逆变换，则求出卷积：$(f*h)_G=U\begin{pmatrix}\hat{h}(\lambda_1)&&\\&\ddots&\\&&\hat{h}(\lambda_n)\end{pmatrix}U^Tf$

很多论文中的Graph卷积公式为$(f*h)_G=U((U^Th)\odot (U^Tf))$，其中$\odot$为哈达玛积。